Este trabalho usa a biblioteca do plot.ly com _offline plot_. Tendo o `plot.ly` instalado e atualizado, ele mostra o gráfico no Jupyter sem precisar de credenciais. Mais informações [aqui](https://plot.ly/python/offline/#generating-offline-graphs-within-jupyter-notebook)

In [1]:
from numpy import corrcoef as correlation
from sklearn.decomposition import PCA
from matplotlib import pyplot as plt
import plotly.graph_objs as go
import plotly.plotly as py
from copy import deepcopy
from plotly import tools
import pandas as pd
import numpy as np
import sklearn
import plotly

In [2]:
# inicializa ambiente offline pro plot.ly
plotly.offline.init_notebook_mode(connected=True)

In [3]:
# ignora erros de SSL
import os, ssl
if (not os.environ.get('PYTHONHTTPSVERIFY', '') and
    getattr(ssl, '_create_unverified_context', None)): 
    ssl._create_default_https_context = ssl._create_unverified_context

In [4]:
# permite mais de um output no 'Out' em cada célula
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

# Projeto Final - Introdução a Ciência dos Dados

## League of Legends Ranked Matches

### Índice
* [Introdução](#Introdução)
* [Análise Inicial dos Dados](#Análise-Inicial-dos-Dados)
    * [Análise - Campeões](#Análise---Campeões)
    * [Análise - Partidas](#Análise---Partidas)
    * [Análise - Jogadores](#Análise---Jogadores)
    * [Análise - Times](#Análise---Times)
    * [Análise - Banimentos](#Análise---Banimentos) 
* [Limpeza dos Dados](#Limpeza-dos-Dados)
    * [Renomeando Colunas](#Renomeando-Colunas)
    * [Concatenando Bases Iguais](#Concatenando-Bases-Iguais)
    * [Ajustando Vitória nas Tabelas](#Ajustando-Vitória-nas-Tabelas)
    * [Conferindo Valores](#Conferindo-Valores)
    * [Última Limpeza](#Última-Limpeza)
* [Análise Exploratória](#Análise-Exploratória)
    * [Explorando Valores dos Jogadores](#Explorando-Valores-dos-Jogadores)
    * [Explorando Valores dos Times](#Explorando-Valores-dos-Times)
* [Correlações](#Correlações)
    * [Atributos dos Jogadores](#Atributos-dos-Jogadores)
    * [Atributos dos Times](#Atributos-dos-Times)
* [Perguntas](#Perguntas)
    * [O Que É Mais Importante Para a Vitória?](#O-Que-É-Mais-Importante-Para-a-Vitória?)
    * [Pick or Ban?](#Pick-or-Ban?)
    * [KDA Médio Por Função?](#KDA-Médio-Por-Função?)
* [Conclusão](#Conclusão)

## Introdução

Este relatório é uma análise de dados de uma base sobre o jogo _League of Legends_. A base [League of Legends Ranked Matches](https://www.kaggle.com/paololol/league-of-legends-ranked-matches), retirada do site _Kaggle_, contém informações sobre 180.000 partidas ranqueadas do jogo, situadas em 6 temporadas (_seasons_ 3 a 8), aproximadamente entre os anos 2014 e 2017. Uma partida ranqueada é um modo competitivo de jogo que categoriza os jogadores em ranques, baseados no nível de habilidade. 

Este relatório foi feito em formato mais intuitivo, com foco no processo e na forma de pensar para resolver o trabalho prático. O objetivo inicial é checar como os dados estão dispostos nos arquivos e fazer qualquer alteração necessária. Depois, limpar os dados que possuírem incoerências. O próximo passo será gerar uma análise exploratória e, por fim, responder algumas perguntas interessantes sobre partidas ranqueadas em _League of Legends_. 

## Análise Inicial dos Dados

Aqui vamos carregar os arquivos e verificar como os dados estão formatados.

Obs: Visto o tamanho dos arquivos (alguns com 56 colunas) e a escassez de recursos computacionais, ficou previamente definido quais atributos seriam mais relevantes para o  trabalho prático.

O download dos arquivos é feito automaticamente pelo `read_csv` do pandas, mas caso ele falhe, deixo comentado uma linha com a opção para abrir os arquivos de uma pasta chamada `data`. Seguem links para download:

* Champs: [Link1](https://www.kaggle.com/paololol/league-of-legends-ranked-matches/downloads/champs.csv/9) [Link2](https://homepages.dcc.ufmg.br/~irscunha/league-of-legends-ranked-matches/champs.csv)
* Matches: [Link1](https://www.kaggle.com/paololol/league-of-legends-ranked-matches/downloads/matches.csv/9) [Link2](https://homepages.dcc.ufmg.br/~irscunha/league-of-legends-ranked-matches/matches.csv)
* Participants: [Link1](https://www.kaggle.com/paololol/league-of-legends-ranked-matches/downloads/participants.csv/9) [Link2](https://homepages.dcc.ufmg.br/~irscunha/league-of-legends-ranked-matches/participants.csv)
* Stats1: [Link1](https://www.kaggle.com/paololol/league-of-legends-ranked-matches/downloads/stats1.csv/9) [Link2](https://homepages.dcc.ufmg.br/~irscunha/league-of-legends-ranked-matches/stats1.csv)
* Stats2: [Link1](https://www.kaggle.com/paololol/league-of-legends-ranked-matches/downloads/stats2.csv/9) [Link2](https://homepages.dcc.ufmg.br/~irscunha/league-of-legends-ranked-matches/stats2.csv)
* Teamstats: [Link1](https://www.kaggle.com/paololol/league-of-legends-ranked-matches/downloads/teamstats.csv/9) [Link2](https://homepages.dcc.ufmg.br/~irscunha/league-of-legends-ranked-matches/teamstats.csv)
* Teambans: [Link1](https://www.kaggle.com/paololol/league-of-legends-ranked-matches/downloads/teambans.csv/9) [Link2](https://homepages.dcc.ufmg.br/~irscunha/league-of-legends-ranked-matches/teambans.csv)


### Análise - Campeões

* `name` (inteiro): nome do campeão
* `id` (inteiro): identificador único

In [5]:
# df_champs = pd.read_csv('./data/champs.csv')
df_champs = pd.read_csv('https://homepages.dcc.ufmg.br/~irscunha/league-of-legends-ranked-matches/champs.csv')
df_champs.head()

print('shape:', df_champs.shape)

,name,id
0,Jax,24
1,Sona,37
2,Tristana,18
3,Varus,110
4,Fiora,114


shape: (138, 2)


### Análise - Partidas

* `id` (inteiro): identificador único
* `queueid` (inteiro): identificador de qual fila ranqueada a partida estava
* `seasonid` (inteiro): identificador da temporada

In [6]:
# df_matches = pd.read_csv('./data/matches.csv', usecols=['id', 'queueid', 'seasonid'])
df_matches = pd.read_csv('https://homepages.dcc.ufmg.br/~irscunha/league-of-legends-ranked-matches/matches.csv',
                        usecols=['id', 'queueid', 'seasonid'])
df_matches.head()

print('shape:', df_matches.shape)

,id,queueid,seasonid
0,10,420,8
1,11,420,8
2,12,420,8
3,13,420,8
4,14,420,8


shape: (184069, 3)


### Análise - Jogadores

* `id` (inteiro): identificador único
* `matchid` (inteiro): referencia a tabela de partidas
* `player` (inteiro): 
    * 1-5: time azul
    * 6-10: time vermelho
* `championid` (inteiro): referencia a tabela de campeões
* `role` (string): função que o jogador exerceu
    * `SOLO`
    * `NONE`
    * `DUO_CARRY` 
    * `DUO_SUPPORT`
* `position` (string): parte do mapa onde o jogador estava
    * `BOT` 
    * `JUNGLE` 
    * `TOP`
    * `MID`

In [7]:
# df_players = pd.read_csv('./data/participants.csv', usecols=['id', 'matchid', 'player', 'championid', 'role', 'position'])
df_players = pd.read_csv('https://homepages.dcc.ufmg.br/~irscunha/league-of-legends-ranked-matches/participants.csv',
                        usecols=['id', 'matchid', 'player', 'championid', 'role', 'position'])
df_players.head()

print('shape:', df_players.shape)

,id,matchid,player,championid,role,position
0,9,10,1,19,NONE,JUNGLE
1,10,10,2,267,DUO_SUPPORT,BOT
2,11,10,3,119,DUO_CARRY,BOT
3,12,10,4,114,SOLO,TOP
4,13,10,5,112,SOLO,MID


shape: (1834520, 6)


* `id` (inteiro): identificador único
* `win` (inteiro): jogador ganhou a partida em que participa?
* `kills` (inteiro): abatimentos feitos pelo jogador na partida
* `deaths` (inteiro): mortes na partida
* `assists` (inteiro): assistências em abatimentos na artida
* `visionscore` (inteiro): placar de visão, uma estatística sobre a contribuição em visão que o jogador teve. Geralmente entre 5 e 150 pontos
    * funciona da seguinte forma: cada minuto de sentinela (item que garante visão sobre uma área finita do mapa) colocada gera 1 ponto ao jogador (existem vários modificadores dependendo do tipo de sentinela colocada que podem aumentar esse valor). Toda vez que o jogador retira uma sentinela inimiga, o tempo de vida restante dela é convertida em pontos (1 minuto:1 ponto). Há vários modificadores também que dão pontos bônus base quando usados.
* `totminionskilled` (inteiro): o _farm_ do jogador, tropas inimigas (_minions_) e neutras (monstros da selva) abatidas em troca de ouro e experiência (não conta campeões)

In [8]:
# df_stats1 = pd.read_csv('./data/stats1.csv', usecols=['id', 'win', 'kills', 'deaths', 'assists', 'visionscore', 
#                                                       'totminionskilled']) 
df_stats1 = pd.read_csv('https://homepages.dcc.ufmg.br/~irscunha/league-of-legends-ranked-matches/stats1.csv',
                       usecols=['id', 'win', 'kills', 'deaths', 'assists', 'visionscore', 'totminionskilled'])
df_stats1.head()

print('shape:', df_stats1.shape)

,id,win,kills,deaths,assists,visionscore,totminionskilled
0,9,0,6,10,1,14,42
1,10,0,0,2,12,30,17
2,11,0,7,8,5,26,205
3,12,0,5,11,2,5,164
4,13,0,2,8,2,15,235


shape: (999999, 7)


In [9]:
# df_stats2 = pd.read_csv('./data/stats2.csv', usecols=['id', 'win', 'kills', 'deaths', 'assists', 'visionscore', 
#                                                       'totminionskilled']) 
df_stats2 = pd.read_csv('https://homepages.dcc.ufmg.br/~irscunha/league-of-legends-ranked-matches/stats2.csv',
                       usecols=['id', 'win', 'kills', 'deaths', 'assists', 'visionscore', 'totminionskilled'])
df_stats2.head()

print('shape:', df_stats2.shape)

,id,win,kills,deaths,assists,visionscore,totminionskilled
0,1028382,0,7,5,1,0,131
1,1028383,0,0,0,0,0,0
2,1028384,0,0,0,0,0,0
3,1028385,0,0,0,0,0,11
4,1028386,0,0,0,0,0,11


shape: (834518, 7)


### Análise - Times

* `matchid` (inteiro): referencia tabela de partidas
* `teamid` (inteiro): referencia times
* `firstblood` (inteiro): time pegou o primeiro abatimento da partida? (ganha bônus de ouro)
* `firsttower` (inteiro): o time destruiu a primeira torre da partida? (ganha bônus de ouro)
* `towerkills` (inteiro): número de torres destruídas
* `inhibkills` (inteiro): número de inibidores destruídos
* `baronkills` (inteiro): número de barões feitos
* `dragonkills` (inteiro): número de dragões feitos 
* `harrykills` (inteiro): arauto foi feito nessa partida?

In [10]:
# df_teamstats = pd.read_csv('./data/teamstats.csv', usecols=['matchid', 'teamid', 'firstblood', 'firsttower', 'towerkills',
#                                                             'inhibkills', 'baronkills', 'dragonkills', 'harrykills'])
df_teamstats = pd.read_csv('https://homepages.dcc.ufmg.br/~irscunha/league-of-legends-ranked-matches/teamstats.csv',
                       usecols=['matchid', 'teamid', 'firstblood', 'firsttower', 'towerkills', 'inhibkills', 
                                'baronkills', 'dragonkills', 'harrykills'])
df_teamstats.head()

print('shape:', df_teamstats.shape)

,matchid,teamid,firstblood,firsttower,towerkills,inhibkills,baronkills,dragonkills,harrykills
0,10,100,0,1,5,0,0,0,0
1,10,200,1,0,10,3,1,3,1
2,11,100,1,0,2,0,0,0,0
3,11,200,0,1,10,3,0,2,0
4,12,100,1,0,1,0,0,0,0


shape: (368138, 9)


### Análise - Banimentos

* `matchid` (inteiro): referencia a tabela de partidas
* `teamid` (inteiro): referencia time
* `championid` (inteiro): campeão banido

In [11]:
df_teambans = pd.read_csv('https://homepages.dcc.ufmg.br/~irscunha/league-of-legends-ranked-matches/teambans.csv',
                       usecols=['matchid', 'teamid', 'championid'])
df_teambans.head()

print('shape:', df_teambans.shape)

,matchid,teamid,championid
0,10,100,11
1,10,100,117
2,10,100,120
3,10,200,84
4,10,200,201


shape: (1099185, 3)


## Limpeza dos Dados

Para cada uma das mini-bases vamos renomear colunas, concatenar bases, ordenar os dados, conferir os valores únicos de dados categóricos ou o máximo e mínimo de cada coluna de dados numéricos, substituir valores NaN (se for necessário) e, obviamente, limpar todos os dados incorretos ou incompletos.

### Renomeando Colunas

O nome `id` é muito vago e pode causar confusões, por isso vamos trocar todos por nomes que façam mais sentido em cada tabela:

Por opção pessoal, vamos renomear também a coluna `totminionskilled` para `farm`, que é mais intuitivo.

In [12]:
df_champs.rename(columns={'id': 'championid'}, inplace=True)
print('df_champs:', list(df_champs))

df_matches.rename(columns={'id': 'matchid'}, inplace=True)
print('df_matches:', list(df_matches))

df_players.rename(columns={'id': 'playerid'}, inplace=True)
print('df_players:', list(df_players))

df_stats1.rename(columns={'id': 'playerid', 'totminionskilled': 'farm'}, inplace=True)
print('df_stats1:', list(df_stats1))

df_stats2.rename(columns={'id': 'playerid', 'totminionskilled': 'farm'}, inplace=True)
print('df_stats2:', list(df_stats2))

df_champs: ['name', 'championid']
df_matches: ['matchid', 'queueid', 'seasonid']
df_players: ['playerid', 'matchid', 'player', 'championid', 'role', 'position']
df_stats1: ['playerid', 'win', 'kills', 'deaths', 'assists', 'visionscore', 'farm']
df_stats2: ['playerid', 'win', 'kills', 'deaths', 'assists', 'visionscore', 'farm']


### Concatenando Bases Iguais

Em seguida, vamos concatenar as bases `df_stats1` e `df_stats2`, que possuem os mesmos dados, só foram separadas por serem muito grandes:

In [13]:
df_playerstats = pd.concat([df_stats1, df_stats2])
df_playerstats.head()
df_playerstats.tail()

print('shape:', df_playerstats.shape)

,playerid,win,kills,deaths,assists,visionscore,farm
0,9,0,6,10,1,14,42
1,10,0,0,2,12,30,17
2,11,0,7,8,5,26,205
3,12,0,5,11,2,5,164
4,13,0,2,8,2,15,235


,playerid,win,kills,deaths,assists,visionscore,farm
834513,1865600,0,11,10,2,0,292
834514,1865601,0,11,9,7,0,189
834515,1865602,0,4,8,10,0,59
834516,1865603,0,2,11,3,0,53
834517,1865604,0,7,11,9,0,235


shape: (1834517, 7)


Desejamos também juntar as tabelas do jogadores e de suas estatísticas, mas devemos notar o seguinte fator, o número de linhas na tabela dos jogadores é maior que o de suas estatísticas, isto é, existem jogadores sem informações que nós devemos remover:

In [14]:
print('jogadores antes:', df_players.shape[0])
print('informações:', df_playerstats.shape[0])

lista = list(set(df_players['playerid']) - set(df_playerstats['playerid']))
print('jogadores sem informação:', lista)

df_players = df_players[~df_players.playerid.isin(lista)]
print('jogadores agora:', df_players.shape[0])  

print('concatenando...')
df_players = df_players.join(df_playerstats.set_index('playerid'), on='playerid')

df_players.head()

print('shape:', df_players.shape)

jogadores antes: 1834520
informações: 1834517
jogadores sem informação: [1272124, 1272013, 1272175]
jogadores agora: 1834517
concatenando...


,playerid,matchid,player,championid,role,position,win,kills,deaths,assists,visionscore,farm
0,9,10,1,19,NONE,JUNGLE,0,6,10,1,14,42
1,10,10,2,267,DUO_SUPPORT,BOT,0,0,2,12,30,17
2,11,10,3,119,DUO_CARRY,BOT,0,7,8,5,26,205
3,12,10,4,114,SOLO,TOP,0,5,11,2,5,164
4,13,10,5,112,SOLO,MID,0,2,8,2,15,235


shape: (1834517, 12)


### Ajustando Vitória nas Tabelas

Com as informações dos jogadores todas unificadas na tabela acima, percebemos uma coisa intrigante: por que a informação de vitória está na tabela dos jogadores e não dos times? É algo relacionado à facilidade em lidar com os dados, mas seria bom ter a mesma informação na tabela de times.

Começaremos separando a informação de vitória e o id das partidas e, em seguida, iremos juntar com a tabela `teamstats`:

In [15]:
df = pd.DataFrame({'matchid': df_players.matchid, 'win': df_players.win})

print('valores extraídos da df_players:')
df.head()

df.drop_duplicates(inplace=True)
print('sem linhas duplicadas:')
df.head()


valores extraídos da df_players:


,matchid,win
0,10,0
1,10,0
2,10,0
3,10,0
4,10,0


sem linhas duplicadas:


,matchid,win
0,10,0
5,10,1
10,11,0
15,11,1
20,12,0


Com a informação de vitória separada junto com o id, devemos criar um id único nas duas tabelas a serem unidas (`df` e `df_teamstats`) e concatenar as tabelas baseadas nesse id, supondo que o primeiro time sempre é o número 100 e o segundo é sempre o 200.

Notamos que o número de times é maior que o número de vitórias, então vamos excluir as partidas que não possuem informação de quem ganhou (ou ela está incompleta):

In [16]:
d = df['matchid'].value_counts()
lista = [x for x,y in zip(d.keys().tolist(), d.tolist())  if y < 2]
print('partidas com estatísticas incompletas:', lista)

print('vitórias antes:', df.shape[0])
df = df[~df.matchid.isin(lista)]
print('vitórias agora:', df.shape[0])

print('df_teamstats antes:', df_teamstats.shape[0])
df_teamstats = df_teamstats[~df_teamstats.matchid.isin(lista)]
print('df_teamstats agora:', df_teamstats.shape[0])

print('df_players agora:', df_players.shape[0])
df_players = df_players[~df_players.matchid.isin(lista)]
print('df_players agora:', df_players.shape[0])

print('df_matches agora:', df_matches.shape[0])
df_matches = df_matches[~df_matches.matchid.isin(lista)]
print('df_matches agora:', df_matches.shape[0])

partidas com estatísticas incompletas: [127721, 127704]
vitórias antes: 368136
vitórias agora: 368134
df_teamstats antes: 368138
df_teamstats agora: 368134
df_players agora: 1834517
df_players agora: 1834515
df_matches agora: 184069
df_matches agora: 184067


In [17]:
df['index'] = range(0, len(df))
df_teamstats['index'] = range(0, len(df_teamstats))

df_teamstats = df_teamstats.merge(df.set_index('index'), on=['index', 'matchid'], how='left')
df_teamstats.head()

,matchid,teamid,firstblood,firsttower,towerkills,inhibkills,baronkills,dragonkills,harrykills,index,win
0,10,100,0,1,5,0,0,0,0,0,0.0
1,10,200,1,0,10,3,1,3,1,1,1.0
2,11,100,1,0,2,0,0,0,0,2,0.0
3,11,200,0,1,10,3,0,2,0,3,1.0
4,12,100,1,0,1,0,0,0,0,4,0.0


Agora precisamos limpar as tabelas, da seguinte forma:
* remover `index` da tabela `df_teamstats`
* reorganizar as colunas para melhorar legibilidade

Obs: apesar de ser redundância, a coluna `win` será mantida em `df_players` para facilitar análises posteriores.

In [18]:
del df_teamstats['index']

print('sem as colunas:')
df_players.head()
df_teamstats.head()

sem as colunas:


,playerid,matchid,player,championid,role,position,win,kills,deaths,assists,visionscore,farm
0,9,10,1,19,NONE,JUNGLE,0,6,10,1,14,42
1,10,10,2,267,DUO_SUPPORT,BOT,0,0,2,12,30,17
2,11,10,3,119,DUO_CARRY,BOT,0,7,8,5,26,205
3,12,10,4,114,SOLO,TOP,0,5,11,2,5,164
4,13,10,5,112,SOLO,MID,0,2,8,2,15,235


,matchid,teamid,firstblood,firsttower,towerkills,inhibkills,baronkills,dragonkills,harrykills,win
0,10,100,0,1,5,0,0,0,0,0.0
1,10,200,1,0,10,3,1,3,1,1.0
2,11,100,1,0,2,0,0,0,0,0.0
3,11,200,0,1,10,3,0,2,0,1.0
4,12,100,1,0,1,0,0,0,0,0.0


In [19]:
df_teamstats = df_teamstats.reindex(['matchid', 'teamid', 'win', 'firstblood', 'firsttower', 'towerkills', 'inhibkills', 
                                     'baronkills', 'dragonkills', 'harrykills'], axis=1)
df_teamstats.head()

,matchid,teamid,win,firstblood,firsttower,towerkills,inhibkills,baronkills,dragonkills,harrykills
0,10,100,0.0,0,1,5,0,0,0,0
1,10,200,1.0,1,0,10,3,1,3,1
2,11,100,0.0,1,0,2,0,0,0,0
3,11,200,1.0,0,1,10,3,0,2,0
4,12,100,0.0,1,0,1,0,0,0,0


### Conferindo Valores

Gostaríamos de ver, para cada tabela, quais são os valores únicos discretos, quais são os limites para cada atributo, se existem valores errados/NaN a serem removidos ou substituídos e se existem valores duplicados. Para isso, vamos criando pequenos dataframes para mostrar as informações básicas interessantes sobre os pedaços da base:

In [20]:
df = pd.DataFrame({'df_champs': ['championid', 'name'], 
                   'valores duplicados?': [any(df_champs['championid'].duplicated()), 
                                           any(df_champs['name'].duplicated())],
                   'valores NaN?': [df_champs['championid'].isnull().values.any(), 
                                    df_champs['name'].isnull().values.any()],
                   'mesmo tipo?': [all(isinstance(x, int) for x in df_champs.championid),
                                   all(isinstance(x, str) for x in df_champs.name)]
                  })
df

,df_champs,valores duplicados?,valores NaN?,mesmo tipo?
0,championid,False,False,True
1,name,False,False,True


In [21]:
df = pd.DataFrame({'df_matches': ['matchid', 'queueid', 'seasonid'], 
                   'valores duplicados?': [any(df_matches['matchid'].duplicated()), '-', '-'],
                   'valores NaN?': [df_matches['matchid'].isnull().values.any(), 
                                    df_matches['queueid'].isnull().values.any(), 
                                    df_matches['seasonid'].isnull().values.any()],
                   'valores únicos': ['-', df_matches.queueid.unique(), 
                                      df_matches.seasonid.unique()],
                   'mesmo tipo?': [all(isinstance(x, int) for x in df_matches.matchid),
                                   all(isinstance(x, int) for x in df_matches.queueid),
                                   all(isinstance(x, int) for x in df_matches.seasonid),]
                  })

df

,df_matches,valores duplicados?,valores NaN?,valores únicos,mesmo tipo?
0,matchid,False,False,-,True
1,queueid,-,False,"[420, 440, 410, 4, 42, 9, 41]",True
2,seasonid,-,False,"[8, 7, 6, 5, 4, 3]",True


In [22]:
df = pd.DataFrame({'df_players': ['playerid', 'matchid', 'player', 'championid', 'role', 'position', 'win', 'kills', 
                                  'deaths', 'assists', 'visionscore', 'farm'], 
                   'valores duplicados?': [any(df_players['playerid'].duplicated()), '-', '-', '-', '-', '-', '-', '-', 
                                           '-', '-', '-', '-'],
                   'valores NaN?': [df_players['playerid'].isnull().values.any(), 
                                    df_players['matchid'].isnull().values.any(), 
                                    df_players['player'].isnull().values.any(), 
                                    df_players['championid'].isnull().values.any(),
                                    df_players['role'].isnull().values.any(), 
                                    df_players['position'].isnull().values.any(),
                                    df_players['win'].isnull().values.any(), 
                                    df_players['kills'].isnull().values.any(), 
                                    df_players['deaths'].isnull().values.any(),
                                    df_players['assists'].isnull().values.any(),
                                    df_players['visionscore'].isnull().values.any(),
                                    df_players['farm'].isnull().values.any()],
                   'valores únicos': ['-', '-', df_players.player.unique(), '-', df_players.role.unique(), 
                                      df_players.position.unique(), df_players.win.unique(), '-', '-', '-', '-', '-'],
                   'mínimo': ['-', '-', '-', '-', '-', '-', '-', min(df_players.kills), min(df_players.deaths), 
                              min(df_players.assists), min(df_players.visionscore), min(df_players.farm),],
                   'máximo': ['-', '-', '-', '-', '-', '-', '-', max(df_players.kills), max(df_players.deaths), 
                              max(df_players.assists), max(df_players.visionscore), max(df_players.farm)],
                   'mesmo tipo?': [all(isinstance(x, int) for x in df_players.playerid),
                                   all(isinstance(x, int) for x in df_players.matchid),
                                   all(isinstance(x, int) for x in df_players.player),
                                   all(isinstance(x, int) for x in df_players.championid),
                                   all(isinstance(x, str) for x in df_players.role),
                                   all(isinstance(x, str) for x in df_players.position),
                                   all(isinstance(x, int) for x in df_players.win),
                                   all(isinstance(x, int) for x in df_players.kills),
                                   all(isinstance(x, int) for x in df_players.deaths),
                                   all(isinstance(x, int) for x in df_players.assists),
                                   all(isinstance(x, int) for x in df_players.visionscore),
                                   all(isinstance(x, int) for x in df_players.farm)]
                  })

df

,df_players,valores duplicados?,valores NaN?,valores únicos,mínimo,máximo,mesmo tipo?
0,playerid,False,False,-,-,-,True
1,matchid,-,False,-,-,-,True
2,player,-,False,"[1, 2, 3, 4, 5, 6, 7, 8, 9, 10]",-,-,True
3,championid,-,False,-,-,-,True
4,role,-,False,"[NONE, DUO_SUPPORT, DUO_CARRY, SOLO, DUO]",-,-,True
5,position,-,False,"[JUNGLE, BOT, TOP, MID]",-,-,True
6,win,-,False,"[0, 1]",-,-,True
7,kills,-,False,-,0,45,True
8,deaths,-,False,-,0,38,True
9,assists,-,False,-,0,57,True


In [23]:
df = pd.DataFrame({'df_teamstats': ['matchid', 'teamid', 'win', 'firstblood', 'firsttower', 'towerkills', 'inhibkills', 
                                    'baronkills', 'dragonkills', 'harrykills'], 
                   'valores NaN?': [df_teamstats['matchid'].isnull().values.any(), 
                                    df_teamstats['teamid'].isnull().values.any(),
                                    df_teamstats['win'].isnull().values.any(),
                                    df_teamstats['firstblood'].isnull().values.any(),
                                    df_teamstats['firsttower'].isnull().values.any(), 
                                    df_teamstats['towerkills'].isnull().values.any(),
                                    df_teamstats['inhibkills'].isnull().values.any(), 
                                    df_teamstats['baronkills'].isnull().values.any(),
                                    df_teamstats['dragonkills'].isnull().values.any(),
                                    df_teamstats['harrykills'].isnull().values.any()],
                   'valores únicos': ['-', '-', df_teamstats.win.unique(), df_teamstats.firstblood.unique(), 
                                      df_teamstats.firsttower.unique(), '-', '-', '-', '-', 
                                      df_teamstats.harrykills.unique()],
                   'mínimo': ['-', '-', '-', '-', '-', min(df_teamstats.towerkills), min(df_teamstats.inhibkills), 
                              min(df_teamstats.baronkills), min(df_teamstats.dragonkills), '-'],
                   'máximo': ['-', '-', '-', '-', '-', max(df_teamstats.towerkills), max(df_teamstats.inhibkills), 
                              max(df_teamstats.baronkills), max(df_teamstats.dragonkills), '-'],
                   'mesmo tipo?': [all(isinstance(x, int) for x in df_teamstats.matchid),
                                   all(isinstance(x, int) for x in df_teamstats.teamid),
                                   all(isinstance(x, int) for x in df_teamstats.win),
                                   all(isinstance(x, int) for x in df_teamstats.firstblood),
                                   all(isinstance(x, int) for x in df_teamstats.firsttower),
                                   all(isinstance(x, int) for x in df_teamstats.towerkills),
                                   all(isinstance(x, int) for x in df_teamstats.inhibkills),
                                   all(isinstance(x, int) for x in df_teamstats.baronkills),
                                   all(isinstance(x, int) for x in df_teamstats.dragonkills),
                                   all(isinstance(x, int) for x in df_teamstats.harrykills)]
                  })

df

,df_teamstats,valores NaN?,valores únicos,mínimo,máximo,mesmo tipo?
0,matchid,False,-,-,-,True
1,teamid,False,-,-,-,True
2,win,True,"[0.0, 1.0, nan]",-,-,False
3,firstblood,False,"[0, 1]",-,-,True
4,firsttower,False,"[1, 0]",-,-,True
5,towerkills,False,-,0,16,True
6,inhibkills,False,-,0,13,True
7,baronkills,False,-,0,5,True
8,dragonkills,False,-,0,7,True
9,harrykills,False,"[0, 1, 2]",-,-,True


In [24]:
df = pd.DataFrame({'df_teambans': ['matchid', 'teamid', 'championid'], 
                   'valores NaN?': [df_teambans['matchid'].isnull().values.any(), 
                                    df_teambans['teamid'].isnull().values.any(), 
                                    df_teambans['championid'].isnull().values.any()],
                   'valores únicos': ['-', df_teambans.teamid.unique(), '-'],
                   'mesmo tipo?': [all(isinstance(x, int) for x in df_teambans.matchid),
                                   all(isinstance(x, int) for x in df_teambans.teamid),
                                   all(isinstance(x, int) for x in df_teambans.championid),]
                  })

df

,df_teambans,valores NaN?,valores únicos,mesmo tipo?
0,matchid,False,-,True
1,teamid,False,"[100, 200]",True
2,championid,False,-,True


### Última Limpeza

Até aqui nós percebemos pequenos problemas que tentaremos excluir agora, sendo considerados inconsistências:
1. Os valores na coluna `queueid` na tabela `df_matches` estão, em sua maioria, desatualizados:
    * todos os valores 4 foram substituídos por 420 e todos os valores 9 foram substituídos por 470 na API oficial da Riot Games
    * os valores 41, 42 e 410 foram depreciados na API e serão excluídos da base
    * os valores 470 se referem a outro mapa que não será explorado nesse relatório então eles serão excluídos (consequentemente vamos excluir os valores 9 também)
    * serão então mantidos os valores: 4, 420 e 440

**Obs:** O motivo para a exclusão desses valores é que são estilos diferentes de jogo e mapa, que podem influenciar nas estatísticas aqui obtidas (por exemplo, o mapa Twisted Treeline, que vamos excluir, é 3x3, enquanto o mapa oficial é 5x5; o `queueid` 42 se referia a um estilo de jogo de time fechado, diferente do modo solo/duo, onde você não escolhe a maior parte dos seus companheiros de equipe)

2. Existe um valor na coluna `role` da tabela `df_players` que não condiz com o esperado: o valor `DUO` não informa se a pessoa jogou de `carry` ou `support` e por isso não pode ser considerado nas estatísticas
3. Também iremos remover as linhas que contém absurdos como mais de 1 arauto, mais de 11 torres e mais de 3 inibidores
4. Existem valores NaN na coluna `win` e os demais valores são _float_ sem necessidade
5. Vamos trocar `NONE` porque não é um nome explicativo

1. Removendo valores incorretos no `queueid`:

In [25]:
lista = [x for x,y in zip(df_matches.matchid, df_matches.queueid) if y in [9, 41, 42, 410]]

print('partidas antes:', df_matches.shape[0])
df_matches = df_matches[~df_matches.matchid.isin(lista)]
print('partidas agora:', df_matches.shape[0], '\n')

print('teamstats antes:', df_teamstats.shape[0])
df_teamstats = df_teamstats[~df_teamstats.matchid.isin(lista)]
print('teamstats agora:', df_teamstats.shape[0], '\n')

print('jogadores antes:', df_players.shape[0])
df_players = df_players[~df_players.matchid.isin(lista)]
print('jogadores agora:', df_players.shape[0], '\n')

print('valores únicos da queueid:', df_matches.queueid.unique())

partidas antes: 184067
partidas agora: 177981 

teamstats antes: 368134
teamstats agora: 355962 

jogadores antes: 1834515
jogadores agora: 1779807 

valores únicos da queueid: [420 440   4]


2. Removendo valores incorretos no `role`:

In [26]:
lista = [x for x,y in zip(df_players.playerid, df_players.role) if y == 'DUO']

print('jogadores antes:', df_players.shape[0])
df_players = df_players[~df_players.playerid.isin(lista)]
print('jogadores agora:', df_players.shape[0], '\n')

print('valores únicos da role:', df_players.role.unique())

jogadores antes: 1779807
jogadores agora: 1721459 

valores únicos da role: ['NONE' 'DUO_SUPPORT' 'DUO_CARRY' 'SOLO']


3. Removendo valores incorretos no `towerkills`, no `inhibkills` e no `harrykills`:

In [27]:
lista = [w for w,x,y,z in zip(df_teamstats.matchid, df_teamstats.towerkills, df_teamstats.inhibkills, 
                              df_teamstats.harrykills) if x > 11 or y > 3 or z > 1]

print('partidas antes:', df_matches.shape[0])
df_matches = df_matches[~df_matches.matchid.isin(lista)]
print('partidas agora:', df_matches.shape[0], '\n')

print('teamstats antes:', df_teamstats.shape[0])
df_teamstats = df_teamstats[~df_teamstats.matchid.isin(lista)]
print('teamstats agora:', df_teamstats.shape[0], '\n')

print('jogadores antes:', df_players.shape[0])
df_players = df_players[~df_players.matchid.isin(lista)]
print('jogadores agora:', df_players.shape[0], '\n')

print('mínimo e máximo de towerkills', min(df_teamstats.towerkills), max(df_teamstats.towerkills))
print('mínimo e máximo de inhibkills', min(df_teamstats.inhibkills), max(df_teamstats.inhibkills))
print('valores únicos de harrykills:', df_teamstats.harrykills.unique())

partidas antes: 177981
partidas agora: 163180 

teamstats antes: 355962
teamstats agora: 326360 

jogadores antes: 1721459
jogadores agora: 1576547 

mínimo e máximo de towerkills 0 11
mínimo e máximo de inhibkills 0 3
valores únicos de harrykills: [0 1]


4. Removendo NaN no `win` e passar tipo para inteiro:

In [28]:
index = df_teamstats.loc[df_teamstats['win'].isnull()].index
lista = df_teamstats.iloc[index]['matchid'].values

df_teamstats = df_teamstats.dropna()

print('partidas antes:', df_matches.shape[0])
df_matches = df_matches[~df_matches.matchid.isin(lista)]
print('partidas agora:', df_matches.shape[0], '\n')

print('teamstats antes:', df_teamstats.shape[0])
df_teamstats = df_teamstats[~df_teamstats.matchid.isin(lista)]
print('teamstats agora:', df_teamstats.shape[0], '\n')

print('jogadores antes:', df_players.shape[0])
df_players = df_players[~df_players.matchid.isin(lista)]
print('jogadores agora:', df_players.shape[0], '\n')

df_teamstats['win'] = df_teamstats['win'].astype(int)
print('valores únicos de win:', df_teamstats.win.unique())

partidas antes: 163180
partidas agora: 163167 

teamstats antes: 326347
teamstats agora: 326321 

jogadores antes: 1576547
jogadores agora: 1576417 

valores únicos de win: [0 1]


5. Trocar `NONE` por `JUNGLER`: 

In [29]:
df_players['role'].replace(['NONE'], 'JUNGLER', inplace=True)
df_players.head()

,playerid,matchid,player,championid,role,position,win,kills,deaths,assists,visionscore,farm
0,9,10,1,19,JUNGLER,JUNGLE,0,6,10,1,14,42
1,10,10,2,267,DUO_SUPPORT,BOT,0,0,2,12,30,17
2,11,10,3,119,DUO_CARRY,BOT,0,7,8,5,26,205
3,12,10,4,114,SOLO,TOP,0,5,11,2,5,164
4,13,10,5,112,SOLO,MID,0,2,8,2,15,235


## Análise Exploratória

A ideia aqui é: ver a distribuição de alguns atributos, suas estatísticas básicas e dar algumas explicações para seus valores; visualizar uma matriz de correlação entre todos os atributos e entre todos os atributos vs vitória; tentar reduzir a dimensionalidade nas bases de jogadores e times.

### Explorando Valores dos Jogadores

Vamos começar averiguando os atributos mais interessantes dos jogadores: `role`, `kills`, `deaths`, `assists`, `visionscore`, `farm`.

In [30]:
keys_role = df_players['role'].value_counts().keys().tolist()
counts_role = df_players['role'].value_counts().tolist()
total_role = sum(counts_role)
rates_role = [x/total_role for x in counts_role]

keys_kills = df_players['kills'].value_counts().keys().tolist()
counts_kills = df_players['kills'].value_counts().tolist()

keys_deaths = df_players['deaths'].value_counts().keys().tolist()
counts_deaths = df_players['deaths'].value_counts().tolist()

keys_assists = df_players['assists'].value_counts().keys().tolist()
counts_assists = df_players['assists'].value_counts().tolist()

trace1 = go.Bar(x=keys_role, y=counts_role, name='role', text=['porcentagem: ' + str(round(x,4)) for x in rates_role])
trace2 = go.Bar(x=keys_kills, y=counts_kills, name='kills')
trace3 = go.Bar(x=keys_deaths, y=counts_deaths, name='deaths')
trace4 = go.Bar(x=keys_assists, y=counts_assists, name='assists')

fig = tools.make_subplots(rows=2, cols=2, subplot_titles=('Subplot 1', 'Subplot 2', 'Subplot 3', 'Subplot 4'))

fig.append_trace(trace1, 1, 1)
fig.append_trace(trace2, 1, 2)
fig.append_trace(trace3, 2, 1)
fig.append_trace(trace4, 2, 2)

fig['layout'].update(title='Gráficos de distribuição')

plotly.offline.iplot(figure_or_data=fig)
# py.iplot(fig, filename='plot1')

This is the format of your plot grid:
[ (1,1) x1,y1 ]  [ (1,2) x2,y2 ]
[ (2,1) x3,y3 ]  [ (2,2) x4,y4 ]



Vamos começar vendo a distribuição dos valores de `role`. Na teoria, cada partida possui 5 funções sempre. Como na base as funções de _top laner_ e _mid laner_ são representadas como `SOLO` nós iremos considerar como a mesma função em posições diferentes. Assim, cada função dentre _jungler_, _carry_ e _support_ deveria corresponder à `1/5 (20%)` da base, e a função _solo_ deveria corresponder à `2/5 (40%)` da base. O que corresponde, aproximadamente, ao gráfico mostrado no subplot 1.

Vamos agora conferir algumas estatísticas básicas:

In [31]:
print('kills:')
print('mínimo e máximo:', min(df_players.kills), max(df_players.kills))
print('média:', np.mean(df_players.kills))
print('mediana:', np.median(df_players.kills))
print('variância:', np.var(df_players.kills))
print('desvio-padrão:', np.std(df_players.kills))

kills:
mínimo e máximo: 0 44
média: 5.744466089873428
mediana: 5.0
variância: 20.410269481990824
desvio-padrão: 4.517772623980851


In [32]:
print('deaths:')
print('mínimo e máximo:', min(df_players.deaths), max(df_players.deaths))
print('média:', np.mean(df_players.deaths))
print('mediana:', np.median(df_players.deaths))
print('variância:', np.var(df_players.deaths))
print('desvio-padrão:', np.std(df_players.deaths))

deaths:
mínimo e máximo: 0 38
média: 5.726463873454803
mediana: 6.0
variância: 9.961873227386882
desvio-padrão: 3.15624353106456


In [33]:
print('assists:')
print('mínimo e máximo:', min(df_players.assists), max(df_players.assists))
print('média:', np.mean(df_players.assists))
print('mediana:', np.median(df_players.assists))
print('variância:', np.var(df_players.assists))
print('desvio-padrão:', np.std(df_players.assists))

assists:
mínimo e máximo: 0 52
média: 8.153088300874705
mediana: 7.0
variância: 33.152296662189805
desvio-padrão: 5.7578031107523815


Falando um pouco sobre os três conjuntos de estatísticas para `kills`, `deaths` e `assists`: a primeira coisa que notamos é que os valores são bastante próximos, provavelmente porque são vários jogadores na mesma partida e as estatísticas de um influenciam nas estatísticas de outros. De cara percebemos que o número médio de assists é maior que de _kills_ e _deaths_ e isso pode ser explicado pela facilidade em se conseguir uma assistência: basta causar algum dano no campeão inimigo enquanto o mesmo estiver em luta e for morto nela.

Notamos também que a variância em `kills` e em `assists` é bem maior que em `deaths`: creio que alguns jogadores preferem jogar solo (diminuindo o número de assistências) enquanto outros preferem juntar nas lutas e/ou não consiguem dar o último _hit_ em campeões inimigos (diminuindo o número de abates).

In [34]:
keys_visionscore = df_players['visionscore'].value_counts().keys().tolist()
counts_visionscore = df_players['visionscore'].value_counts().tolist()

keys_totminionskilled = df_players['farm'].value_counts().keys().tolist()
counts_totminionskilled = df_players['farm'].value_counts().tolist()

trace1 = go.Bar(x=keys_visionscore, y=counts_visionscore, name='visionscore')
trace2 = go.Bar(x=keys_totminionskilled, y=counts_totminionskilled, name='farm')

fig = tools.make_subplots(rows=2, cols=1, subplot_titles=('Subplot 1', 'Subplot 2'))

fig.append_trace(trace1, 1, 1)
fig.append_trace(trace2, 2, 1)

fig['layout'].update(title='Gráficos de distribuição')

plotly.offline.iplot(figure_or_data=fig)
# py.iplot(fig, filename='plot2')

This is the format of your plot grid:
[ (1,1) x1,y1 ]
[ (2,1) x2,y2 ]



In [35]:
print('visionscore:')
print('mínimo e máximo:', min(df_players.visionscore), max(df_players.visionscore))
print('média:', np.mean(df_players.visionscore))
print('mediana:', np.median(df_players.visionscore))
print('variância:', np.var(df_players.visionscore))
print('desvio-padrão:', np.std(df_players.visionscore))

visionscore:
mínimo e máximo: 0 175
média: 14.078259115449782
mediana: 10.0
variância: 290.7411822163572
desvio-padrão: 17.05113433811244


In [36]:
print('farm:')
print('mínimo e máximo:', min(df_players.farm), max(df_players.farm))
print('média:', np.mean(df_players.farm))
print('mediana:', np.median(df_players.farm))
print('variância:', np.var(df_players.farm))
print('desvio-padrão:', np.std(df_players.farm))

farm:
mínimo e máximo: 0 591
média: 119.89593616409871
mediana: 126.0
variância: 6727.67504698034
desvio-padrão: 82.02240576196445


Análisando agora os valores de `visionscore` e `farm`. Aqui a diferença de valores em cada atributo é gritante e pode ser explicada pelas características pessoais dos jogadores e de suas funções exercidas (já que não são atributos influenciados por fatores externos), além do nível de habilidade. 

No caso do placar de visão, infelizmente em ranques mais baixos os jogadores não constumam colocar sentinelas de visão no mapa (apesar de visão ser um dos objetivos mais importantes do jogo) e a função de colocar sentinelas fica encabida aos suportes, por isso temos um pico no valor 0. 

Para o total de _farm_ do jogador, precisamos levar em consideração que diferentes funções possuem estilo de _farm_ diferente. Suportes geralmente farmam pouco, em torno de 50 por partida. As outras funções deveriam abater em torno de 200 tropas por partida em tempo normal (20~40 minutos de jogo), mas não é o que geralmente acontece, como podemos notar no gráfico. Em ranques mais baixos é comum as pessoas não saberem farmar e existe a concepção de que matar inimigos é mais importante que conseguir dinheiro e experiência, o que é ridiculamente errôneo (chega um ponto que os campeões podem valer menos que tropas e objetivos). 

É curioso notar que o maior _farm_ é 591. Algumas estratégias fortes que existiram nos metas (mudança em alguma característica do jogo que fortalece ou enfraquecem determinado aspecto do jogo) englobados na base que priorizam o _farm_, geralmente em conjunto com levar torres/objetivos rapidamente. A título de curiosidade, algumas das estratégias mais famosas são: funilamento de recursos (Karthus e Nunu, Master Yi e Taric, dentre outras combinações especializadas em andar sempre em dupla e concentrar experiência e ouro em um único campeão), _proxy Singed_ (Singed mata os minions entre as torres na rota do topo, fazendo com que sua tropa puxe mais rápido as torres e evitando conflitos com campeão inimigo), _tower rush_ (foco exclusivamente em tropas e torres, geralmente utilizando Tryndamere, Yorick, Ziggs e Heimerdinger).

### Explorando Valores dos Times

Vamos começar averiguando os atributos mais interessantes dos times: `towerkills`, `inhibkills`, `baronkills`, `dragonkills`:  

In [37]:
keys_towers = df_teamstats['towerkills'].value_counts().keys().tolist()
counts_towers = df_teamstats['towerkills'].value_counts().tolist()

keys_inhibs = df_teamstats['inhibkills'].value_counts().keys().tolist()
counts_inhibs = df_teamstats['inhibkills'].value_counts().tolist()

keys_barons = df_teamstats['baronkills'].value_counts().keys().tolist()
counts_barons = df_teamstats['baronkills'].value_counts().tolist()

keys_dragons = df_teamstats['dragonkills'].value_counts().keys().tolist()
counts_dragons = df_teamstats['dragonkills'].value_counts().tolist()

trace1 = go.Bar(x=keys_towers, y=counts_towers, name='towerkills')
trace2 = go.Bar(x=keys_inhibs, y=counts_inhibs, name='inhibkills')
trace3 = go.Bar(x=keys_barons, y=counts_barons, name='baronkills')
trace4 = go.Bar(x=keys_dragons, y=counts_dragons, name='dragonskill')

fig = tools.make_subplots(rows=2, cols=2, subplot_titles=('Subplot 1', 'Subplot 2', 'Subplot 3', 'Subplot 4'))

fig.append_trace(trace1, 1, 1)
fig.append_trace(trace2, 1, 2)
fig.append_trace(trace3, 2, 1)
fig.append_trace(trace4, 2, 2)

fig['layout'].update(title='Gráficos de distribuição')

plotly.offline.iplot(figure_or_data=fig)
# py.iplot(fig, filename='plot3')

This is the format of your plot grid:
[ (1,1) x1,y1 ]  [ (1,2) x2,y2 ]
[ (2,1) x3,y3 ]  [ (2,2) x4,y4 ]



In [38]:
print('towerkills:')
print('mínimo e máximo:', min(df_teamstats.towerkills), max(df_teamstats.towerkills))
print('média:', np.mean(df_teamstats.towerkills))
print('mediana:', np.median(df_teamstats.towerkills))
print('variância:', np.var(df_teamstats.towerkills))
print('desvio-padrão:', np.std(df_teamstats.towerkills))

towerkills:
mínimo e máximo: 0 11
média: 5.648153198844083
mediana: 6.0
variância: 14.662426596884188
desvio-padrão: 3.8291548149538417


In [39]:
print('inhibkills:')
print('mínimo e máximo:', min(df_teamstats.inhibkills), max(df_teamstats.inhibkills))
print('média:', np.mean(df_teamstats.inhibkills))
print('mediana:', np.median(df_teamstats.inhibkills))
print('variância:', np.var(df_teamstats.inhibkills))
print('desvio-padrão:', np.std(df_teamstats.inhibkills))

inhibkills:
mínimo e máximo: 0 3
média: 0.9192604827761621
mediana: 1.0
variância: 1.1363282042529914
desvio-padrão: 1.0659869625154856


In [40]:
print('baronkills:')
print('mínimo e máximo:', min(df_teamstats.baronkills), max(df_teamstats.baronkills))
print('média:', np.mean(df_teamstats.baronkills))
print('mediana:', np.median(df_teamstats.baronkills))
print('variância:', np.var(df_teamstats.baronkills))
print('desvio-padrão:', np.std(df_teamstats.baronkills))

baronkills:
mínimo e máximo: 0 4
média: 0.3855252956444728
mediana: 0.0
variância: 0.3329543308025615
desvio-padrão: 0.5770219500179881


In [41]:
print('dragonkills:')
print('mínimo e máximo:', min(df_teamstats.dragonkills), max(df_teamstats.dragonkills))
print('média:', np.mean(df_teamstats.dragonkills))
print('mediana:', np.median(df_teamstats.dragonkills))
print('variância:', np.var(df_teamstats.dragonkills))
print('desvio-padrão:', np.std(df_teamstats.dragonkills))

dragonkills:
mínimo e máximo: 0 7
média: 1.4018251966621824
mediana: 1.0
variância: 1.4578929119185708
desvio-padrão: 1.2074323632893773


Analisando os valores e gráficos para `towerkills`, `inhibkills`, `baronkills` e `dragonkills`: todos eles são objetivos importantes que fornecem meios reais de ganhar uma partida. Ainda sim, em todos as visualizações pode-se notar que o valor 0 se sobressai, dando a entender que os times não costumam focar em objetivos ou os times que perdem realmente não se esforçaram muito em objetivos. 

O valor de 5 torres em média é o que pode ser melhor explicado, já que esse é o número mínimo de torres derrubadas para conseguir finalizar o jogo (3 torres de uma rota, mais as 2 torres do nexus). O valor mediano de inibidores também está na mesma categoria, já que só há um inibidor por rota.

Apesar de não ser uma surpresa, os valores em relação a barões e dragões são decepcionantes. O número de times que não fizeram um barão sequer é maior que o dobro do número de times que fizeram pelo menos um. Isso nos dá duas indicações que não serão exploradas nesse trabalho, mas seriam análises interessantes:

1. As partidas têm acabado antes de 20 minutos (tempo em que o Barão nasce) ou;
2. Os times não tem interesse em fazer o Barão.

A Riot Games, desenvolvedora de League of Legends, vem alterando o bônus de objetivos como Barão, dragões e Arauto afim de tornar o jogo mais depedente deles e incentivar as equipes no elo mais baixo a focarem em objetivos.  

## Correlações

Gostaríamos agora de conferir os atributos que possuem maior correlação entre si e, consequentemente, checar quais são os atributos que possuem maior correlação com `win`.

### Atributos dos Jogadores


In [42]:
df = df_players[['win', 'kills', 'deaths', 'assists', 'visionscore', 'farm']]

labels = list(df)

cor_matrix = np.zeros((len(labels), len(labels)))

for i in range(len(labels)):
    for j in range(len(labels)):
        if i == j:
            cor_matrix[i, j] = 1
        elif i < j:
            b = correlation(df[labels[i]], df[labels[j]])
            cor_matrix[i, j] = b[0, 1]
            cor_matrix[j, i] = b[0, 1]

In [43]:
trace = go.Heatmap(z=cor_matrix, x=labels, y=labels)
data=[trace]
plotly.offline.iplot(figure_or_data=data)
# py.iplot(data, filename='plot4')

Talvez por serem atributos distintos uns dos outros para um mesmo jogador, as correlações para a tabela são surpreendentemente baixas, em geral. As 4 maiores que podemos notar pelo heatmap são:

* `kills` x `farm` (0.43): faz sentido pois com mais experiência e ouro, fica fácil abater outros jogadores;
* `assists` x `win` (0.32): provavelmente à participação no jogo ajudando outros jogadores a adquirirem vantagem em suas rotas;
* `visionscore` x `assists` (0.31): duas explicações plausíveis: os _roamings_ (sair de sua rota e ir ajudar outras rotas) que alguns jogadores fazem os levam a deixar mais sentilas e ganhar mais assistências e; o ganho de assistências e a cobertura do mapa com sentinelas geralmente são os focos dos suportes; 
* `kills` x `win` (0.27): abater campeões inimigos tira vantagem deles e a deixa consigo; é óbvio que nem sempre os jogadores conseguem utilizar essa vantagem para vencer as partidas (ou não optam por distribuir a vantagem com companheiros de time), mas o abate de campeões é importante para conquistar a vitória. 

É interessante também notar a mais baixa correlação apresentada: `deaths` x `win`. Ela é negativa e explicita o que imaginávamos, suas chances de vencer diminuem à medida que o jogador morre pois ele não só perde grandes quantias de ouro e experiência, como as deixa para o adversário e ainda corre o risco de perder objetivos importantes.

### Atributos dos Times

In [44]:
df = df_teamstats[['firstblood', 'firsttower', 'towerkills', 'inhibkills', 'baronkills', 'dragonkills', 'harrykills']]

labels = list(df)

cor_matrix = np.zeros((len(labels), len(labels)))

for i in range(len(labels)):
    for j in range(len(labels)):
        if i == j:
            cor_matrix[i, j] = 1
        elif i < j:
            b = correlation(df[labels[i]], df[labels[j]])
            cor_matrix[i, j] = b[0, 1]
            cor_matrix[j, i] = b[0, 1]

In [45]:
trace = go.Heatmap(z=cor_matrix, x=labels, y=labels)
data=[trace]
plotly.offline.iplot(figure_or_data=data)
# py.iplot(data, filename='plot5')

Aqui podemos perceber correlações um pouco melhores, sendo a mais baixa 0.06. As 5 melhores correlações foram:

* `inhibkills` x `towerkills` (0.86): como para chegar aos inibidores é necessário levar torres primeiro, faz total sentido;
* `dragonkills` x `towerkills` (0.64) e `inhibkills` x `dragonkills` (0.54): um dos dragões elementais mais cobiçados da buff de dano em torres e inibidores;
* `firsttower` x `towerkills` (0.52): por causa do bônus de primeira torre, se o time souber aproveitar é uma condição para levar mais torres;
* `towerkills` x `baronkills` (0.50): a derrubada de torres significa maior controle do mapa por parte de um time, então podemos explicar essa correlação por facilitar a conquista do barão; 

## Perguntas

Iremos agora responder perguntas interessantes sobre _League of Legends_.

### O Que É Mais Importante Para a Vitória?

Vamos tentar algo diferente: calcular a média entre os atributos de jogadores do mesmo time e correlacionar com os atributos do time.

In [46]:
df = deepcopy(df_players)
df['teamid'] = df['player'].map(lambda x: 100 if 1 <= x <= 5 else 200)

df = df[['matchid', 'teamid', 'win', 'kills', 'deaths', 'assists', 'visionscore', 'farm']]

df = pd.DataFrame(df.groupby(['matchid', 'teamid']).mean())
df.reset_index(inplace=True)
df.head()

,matchid,teamid,win,kills,deaths,assists,visionscore,farm
0,10,100,0.0,4.0,7.8,4.4,18.0,132.6
1,10,200,1.0,7.8,4.0,11.2,27.6,132.2
2,11,100,0.0,6.6,6.4,6.8,16.8,117.4
3,11,200,1.0,6.4,6.6,5.8,21.2,129.6
4,12,100,0.0,4.0,6.4,5.4,12.4,79.0


In [47]:
df = df.merge(df_teamstats.set_index('matchid'), on=['matchid', 'teamid'], how='left')
df = df.dropna()
del df['win_y']
df.rename(columns={'win_x': 'win'}, inplace=True)
df.head()

,matchid,teamid,win,kills,deaths,assists,visionscore,farm,firstblood,firsttower,towerkills,inhibkills,baronkills,dragonkills,harrykills
0,10,100,0.0,4.0,7.8,4.4,18.0,132.6,0.0,1.0,5.0,0.0,0.0,0.0,0.0
1,10,200,1.0,7.8,4.0,11.2,27.6,132.2,1.0,0.0,10.0,3.0,1.0,3.0,1.0
2,11,100,0.0,6.6,6.4,6.8,16.8,117.4,1.0,0.0,2.0,0.0,0.0,0.0,0.0
3,11,200,1.0,6.4,6.6,5.8,21.2,129.6,0.0,1.0,10.0,3.0,0.0,2.0,0.0
4,12,100,0.0,4.0,6.4,5.4,12.4,79.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0


In [48]:
df = df[['win', 'kills', 'deaths', 'assists', 'visionscore', 'farm', 'firstblood', 'firsttower', 'towerkills', 
         'inhibkills', 'baronkills', 'dragonkills', 'harrykills']]

labels = list(df)

cor_matrix = np.zeros((len(labels), len(labels)))

for i in range(len(labels)):
    for j in range(len(labels)):
        if i == j:
            cor_matrix[i, j] = 1
        elif i < j:
            b = correlation(df[labels[i]], df[labels[j]])
            cor_matrix[i, j] = b[0, 1]
            cor_matrix[j, i] = b[0, 1]

In [49]:
trace = go.Heatmap(z=cor_matrix, x=labels, y=labels)
data=[trace]
plotly.offline.iplot(figure_or_data=data)
# py.iplot(data, filename='plot6')

Essa visualização é uma ótima maneira de vermos a correlação geral entre os atributos, utilizando a média dos atributos dos jogadores por times. Também é possível ver de maneira clara a correlação dos atributos com `win`. Em ordem crescente, os atributos mais importantes para vitória são:

1. torres;
2. inibidores;
3. dragões;
4. abates;
5. assistências;
6. bônus de primeira torre;
7. barões;
8. Arauto;
9. bônus de primeiro abate;
10. farm;
11. placar de visão;
12. mortes;

Disso, podemos notar uma surpresa: talvez pela quantidade de valores 0 na base, esta visualização nos mostra que o placar de visão não é tão importante para se vencer uma partida. Talvez se fosse possível dividir a base por elo, teríamos a importância do placar de visão aumentando conforme falamos de elos mais altos. 

### Pick or Ban?

Campeões _pick or ban_ são aqueles que estão sempre aparecendo nas partidas, seja como seleção ou banimento. Por vezes, existe uma regra entre os jogadores de que se o campeão x estiver "open" (não for banido), ele deve aparecer no jogo. A ideia é conferir se há um padrão entre os 20 campeões em todas as temporadas.

In [50]:
keys_picks = df_players['championid'].value_counts().keys().tolist()
counts_picks = df_players['championid'].value_counts().tolist()
total_picks = sum(counts_picks)
rates_picks = [x/total_picks for x in counts_picks]
names_picks = [df_champs.loc[df_champs['championid'] == x]['name'].iloc[0] for x in keys_picks]

keys_bans = df_teambans['championid'].value_counts().keys().tolist()
counts_bans = df_teambans['championid'].value_counts().tolist()
total_bans = sum(counts_bans)
rates_bans = [x/total_bans for x in counts_bans]
names_bans = [df_champs.loc[df_champs['championid'] == x]['name'].iloc[0] for x in keys_bans]

trace1 = go.Bar(x=names_picks[:20], y=rates_picks[:20], text=['total: ' + str(round(x,4)) for x in counts_picks])
trace2 = go.Bar(x=names_bans[:20], y=rates_bans[:20], text=['total: ' + str(round(x,4)) for x in counts_bans])

fig = tools.make_subplots(rows=1, cols=2, subplot_titles=('Seleções', 'Banimentos'))

fig.append_trace(trace1, 1, 1)
fig.append_trace(trace2, 1, 2)

fig['layout'].update(title='Pick or Ban?', showlegend=False)

plotly.offline.iplot(figure_or_data=fig)
# py.iplot(fig, filename='plot7')

This is the format of your plot grid:
[ (1,1) x1,y1 ]  [ (1,2) x2,y2 ]



Podemos ver alguns nomes aparecendo nas duas listas, como:

* Yasuo, que inclusive possui o maior fã clube do jogo, com uma grande quantidade de jogadores que não jogam com outro campeão; tenho a impressão de que ele só não é mais banido porque as pessoas podem indicar qual campeão elas desejam usar antes dos picks e bans;
* Lee Sin;
* Ahri;
* Vayne;
* Fizz;
* Master Yi, dentre outros.


É interessante notar que alguns campeões da lista possuem um conjunto (habilidades + modo de jogo) difícil de ser balanceado, então eles sempre sofrem nerfs ou ajustes como uma tentativa da Riot Games de tirá-los do _pick or ban_. Também percebemos que grande parte dos campeões banidos são da classe Assassino, uma classe do jogo especializada em dar _burst_ (grande quantidade de dano em frações de segundo) em campeões inimigos; a maioria dos jogadores não sabe como lidar com esse tipo de campeão e se frusta quando o assassino já muito forte dá um _insta kill_ (mata sem dar chance de proteção).

Uma coisa engraçada de se notar é que a campeã Xayah (que aparece em ambas as listas) foi lançada pouco tempo antes dessa base ser disponibilizada no Kaggle, juntamente com o campeão Rakan. Ela estava tão forte que em pouco tempo ela foi banida 77 mil vezes, passando campeões que estão inclusos em todas as temporadas nessa base.  

### KDA Médio Por Função?

O KDA (kills/deaths/assists) é uma função dada por: KDA = (K+A) / Max(1,D), onde K é `kills`, A é `assists` e D é `deaths`. Vamos descobrir qual o KDA médio para cada função.

In [51]:
def kdaCalculator(row):
    return (row['kills'] + row['assists'])/max(1, row['deaths'])

df = deepcopy(df_players)
df = df[['role', 'position', 'kills', 'deaths', 'assists']]

df['kda'] = df.apply(lambda x: kdaCalculator(x), axis=1, result_type='expand')
df.head()

df = pd.DataFrame(df.groupby(['position', 'role']).mean())
df.reset_index(inplace=True)

df

print('média de KDA:', df['kda'].mean())

,role,position,kills,deaths,assists,kda
0,JUNGLER,JUNGLE,6,10,1,0.700000
1,DUO_SUPPORT,BOT,0,2,12,6.000000
2,DUO_CARRY,BOT,7,8,5,1.500000
3,SOLO,TOP,5,11,2,0.636364
4,SOLO,MID,2,8,2,0.500000


,position,role,kills,deaths,assists,kda
0,BOT,DUO_CARRY,7.116956,5.843352,7.251279,3.539446
1,BOT,DUO_SUPPORT,2.252965,5.440836,13.248436,4.312698
2,BOT,JUNGLER,2.242308,2.709615,4.205000,1.552800
3,BOT,SOLO,2.791685,3.465024,4.067752,1.651279
4,JUNGLE,JUNGLER,6.247320,5.687617,7.847103,3.569692
5,MID,DUO_CARRY,7.179705,6.332330,6.646061,3.130517
6,MID,DUO_SUPPORT,4.760667,5.180252,6.895514,2.935912
7,MID,JUNGLER,0.341040,0.583815,0.366474,0.295226
8,MID,SOLO,7.217945,5.941676,6.848901,3.429476
9,TOP,DUO_CARRY,6.953177,6.511706,6.859532,3.034785


média de KDA: 2.6222183540973143


Primeiramente, notamos um erro na base que não foi trabalho neste trabalho prático: valores estranhos para `role` em posições incorretas. Como não vamos tratá-los no escopo deste trabalho, vamos considerar como categorias diferentes.

É interessante notar duas coisas sobre o KDA médio por função:
1. Ele é positivo (KDA > 1) em todos os casos, exceto Jungler no mid e no top (que não fazem sentido);
2. Todos as categorias possuem valores bem parecidos de KDA, independente da função exercida, mas algumas posições se mostram mais agressivas que outras, como `DUO_CARRY` (bot) e `SOLO` (mid) em relação à `SOLO` (top).

## Conclusão

Este relatório foi uma análise exploratória dos dados da base _League of Legends Ranked Matches_, retirada do site Kaggle. Seu objetivo principal era mostrar o processo utilizado na limpeza e análise dos dados, bem como analisar aspectos em profundidade sobre o jogo e responder algumas questões interessantes.

As maiores dificuldades com esse trabalho prático foram: 
* Lidar com o tamanho da base (recursos computacionais escassos e Jupyter pesado);
* Fazer uma limpeza efetiva dos dados;
* Trazer ferramentas que de fato fossem mais para análise que para limpeza.

E apesar de problemas com falta de recursos computacionais, um PCA que não funcionou, valores bizarros na base e nas estatísticas (alguns que não foram possíveis explicar), considero este trabalho satisfatório. Mesmo o que não foi usado neste projeto e que foi ensinado em sala teve que ser revisto para saber se sua utilização era viável. O trabalho também aumentou muito o conhecimento em Python, principalmente para geração de gráficos e uso da biblioteca pandas, que serviram para outros trabalhos feitos este semestre em outras disciplinas. Por fim, o projeto me mostrou vários aspectos interessantes sobre _League of Legends_ e se tornou uma análise divertida de fazer.